## 📌 Retention
- Group A: D+2 이내 배송
- Group B: D+2 초과 배송

  → 재구매율 비교

고객 단위
- 고객 키: `customer_unique_id`

기준일(코호트 기준)
- 기준일: 첫 구매일(first_purchase_date)

재구매의 정의
- 재구매 = 첫 구매 이후의 추가 구매(주문)
    - 동일 주문(같은 order_id)은 제외
    - 첫 구매일 미포함 (strictly after)

윈도우 정의
- D+30 재구매 여부: 첫 구매일로부터 30일 이내 (inclusive)
- D+60 재구매 여부: "

실험군(배송경험 그룹) 정의
- Group A: 고객의 첫 구매가 D+2 이내 배송일 경우
- Group B: 고객의 첫 구매가 D+2 초과 배송일 경우

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# DB 연결
conn = sqlite3.connect("olist.db")

In [3]:
# SQLite에 CSV 적재
path = '/content/drive/MyDrive/Data Analysis/prj1-olist-ecommerce'

pd.read_csv(f"{path}/datasets/raw/olist_orders_dataset.csv").to_sql("orders", conn, if_exists="replace", index=False)
pd.read_csv(f"{path}/datasets/raw/olist_customers_dataset.csv").to_sql("customers", conn, if_exists="replace", index=False)
pd.read_csv(f"{path}/datasets/raw/olist_order_payments_dataset.csv").to_sql("payments", conn, if_exists="replace", index=False)

103886

In [4]:
# 각 주문의 주문 고객, 결제액 집계
query = """
  WITH payments_agg AS (
    SELECT
      order_id
      , SUM(payment_value) AS payment_value
    FROM payments
    GROUP BY order_id
  )
  SELECT
    o.order_id
    , o.customer_id
    , c.customer_unique_id
    , o.order_purchase_timestamp
    , o.order_delivered_customer_date
    , o.order_estimated_delivery_date
    ,  p.payment_value
  FROM orders o
  JOIN customers c
    ON o.customer_id = c.customer_id
  LEFT JOIN payments_agg p
    ON o.order_id = p.order_id
  WHERE o.order_status = 'delivered'
  ;
"""

df = pd.read_sql_query(query, conn)

In [5]:
# 연결 close
conn.close()

In [6]:
# 데이터 확인
display(df.sample(5))
display(df.info())

,order_id,customer_id,customer_unique_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,payment_value
52288,5529ec8272cda325d9f9300e9bb09d5d,31f0bfed6ed789da8198c1478815e537,8f57bc700d1417596da165432439c6cd,2018-06-10 11:48:43,2018-06-22 22:18:35,2018-07-11 00:00:00,223.32
19162,207d35e4a69118f2f9422aa1949043f7,09becaeb993ca4d7a5c0ea6eb2afce30,07c0bb79a7ed118f2738817d8c2fb7b0,2018-05-29 10:19:05,2018-06-12 17:52:44,2018-07-12 00:00:00,40.13
7431,be1d612da3d925938a4706d808b7ef53,916022b684a43cb980350c4557eb0f1e,568875011286e624d148e315c2edf387,2017-05-21 18:23:37,2017-05-23 13:36:35,2017-06-02 00:00:00,45.62
40998,e3770527e77ed1a6d8b5992c286dc4ea,9256fbcdfba18f428fd2325633fb38cf,1159f6ae6d9035b5da1967ebf1b2a6f9,2017-12-27 20:58:14,2018-01-05 21:09:00,2018-01-16 00:00:00,157.30
37435,fd867bdd8258af11c6d8d6ff8c6c82db,ae534112af536deb8446fb273796316d,552a038a90f9896d8d9c4d86de8eeb93,2017-03-23 21:36:05,2017-03-29 13:04:48,2017-04-13 00:00:00,35.52


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96478 entries, 0 to 96477
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       96478 non-null  object 
 1   customer_id                    96478 non-null  object 
 2   customer_unique_id             96478 non-null  object 
 3   order_purchase_timestamp       96478 non-null  object 
 4   order_delivered_customer_date  96470 non-null  object 
 5   order_estimated_delivery_date  96478 non-null  object 
 6   payment_value                  96477 non-null  float64
dtypes: float64(1), object(6)
memory usage: 5.2+ MB


None

In [7]:
# timestamp 타입 변환
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

In [8]:
# 핵심 컬럼 결측 검증
key_cols = ['order_purchase_timestamp', 'order_delivered_customer_date','order_estimated_delivery_date', 'payment_value']
print(df[key_cols].isna().mean().sort_values(ascending=False))

order_delivered_customer_date    0.000083
payment_value                    0.000010
order_purchase_timestamp         0.000000
order_estimated_delivery_date    0.000000
dtype: float64


In [9]:
# 분석 불가능한 row 제거
df = df[df['order_delivered_customer_date'].notna()]
df = df[df['payment_value'].notna()]

In [10]:
# 파생 컬럼 재생성
df['estimated_deadline'] = df['order_estimated_delivery_date'] + pd.Timedelta('23:59:59')

df['delay_sec'] = (df['order_delivered_customer_date'] - df['estimated_deadline']).dt.total_seconds()

df['delay_days'] = np.where(
    df['delay_sec'] > 0
    , np.ceil(df['delay_sec'] / 86400)
    , np.where(
        df['delay_sec'] < 0
        , (df['delay_sec'] / 86400).astype(int)
        , 0
    )
).astype(int)

In [11]:
# 구매일 파생 컬럼 생성
df['purchase_dt'] = df['order_purchase_timestamp'].dt.date

In [12]:
# 고객별 첫 주문 찾기
first = (df.sort_values(['customer_unique_id','purchase_dt','order_id'])
           .groupby('customer_unique_id', as_index=False)
           .first()[['customer_unique_id','order_id','purchase_dt','delay_days']])

first = first.rename(columns={
    'order_id':'first_order_id'
    , 'purchase_dt':'first_purchase_dt'
    , 'delay_days':'first_delay_days'
    })

# 그룹 A, B 라벨
first['group'] = np.where(first['first_delay_days'] <= 2, 'A_≤2d', 'B_>2d')

In [13]:
# 주문 테이블에 first_purchase_dt 붙이기
df2 = df.merge(first[['customer_unique_id','first_purchase_dt','first_order_id','group']], on='customer_unique_id', how='left')

# 첫 주문 자체는 제외 (strictly after)
df2 = df2[df2['order_id'] != df2['first_order_id']].copy()

# day_diff 계산
df2['day_diff'] = (pd.to_datetime(df2['purchase_dt']) - pd.to_datetime(df2['first_purchase_dt'])).dt.days

# D+30 / D+60 윈도우 안에 주문이 있는지 (inclusive)
rep = (df2[df2['day_diff'] >= 0]
         .groupby('customer_unique_id')
         .agg(repurchase_d30=('day_diff', lambda x: int((x <= 30).any()))
              , repurchase_d60=('day_diff', lambda x: int((x <= 60).any())))
         .reset_index())

# 고객 단위 마스터 df
cust = first.merge(rep, on='customer_unique_id', how='left')
cust[['repurchase_d30','repurchase_d60']] = cust[['repurchase_d30','repurchase_d60']].fillna(0).astype(int)

In [14]:
# 그룹별 retention summary
summary = (cust.groupby('group')
              .agg(n_customers=('customer_unique_id','nunique'),
                   d30_rate=('repurchase_d30','mean'),
                   d60_rate=('repurchase_d60','mean'))
              .reset_index())

summary['d30_rate_pct'] = summary['d30_rate'] * 100
summary['d60_rate_pct'] = summary['d60_rate'] * 100

display(summary)

,group,n_customers,d30_rate,d60_rate,d30_rate_pct,d60_rate_pct
0,A_≤2d,88327,0.015228,0.018398,1.522751,1.839755
1,B_>2d,5022,0.014138,0.017324,1.413779,1.732378


In [15]:
# 그룹별, 윈도우별 retention rate 계산
a30 = summary.loc[summary['group']=='A_≤2d','d30_rate_pct'].iloc[0]
b30 = summary.loc[summary['group']=='B_>2d','d30_rate_pct'].iloc[0]
diff_30_pp = (a30 - b30)

a60 = summary.loc[summary['group']=='A_≤2d','d60_rate_pct'].iloc[0]
b60 = summary.loc[summary['group']=='B_>2d','d60_rate_pct'].iloc[0]
diff_60_pp = (a60 - b60)

print(f"D+2 초과 배송을 경험한 고객(Group B)은 D+30 재구매율이 {diff_30_pp:.1f}%p 낮아짐")
print(f"D+2 초과 배송을 경험한 고객(Group B)은 D+60 재구매율이 {diff_60_pp:.1f}%p 낮아짐")

D+2 초과 배송을 경험한 고객(Group B)은 D+30 재구매율이 0.1%p 낮아짐
D+2 초과 배송을 경험한 고객(Group B)은 D+60 재구매율이 0.1%p 낮아짐


#### ✅ Retention 관점에서의 임계점 재탐색 필요
배송 지연 2일은 고객 만족도(리뷰 평점) 하락이 시작되는 임계점이었으나<br>
해당 만족도 하락이 재구매 행동으로 즉각 전이되지는 않음 (전체 고객 평균 기준)<br>
-> 세분화/임계점 재설정이 필요하다는 근거

In [16]:
# delay_days 분포 확인 (bucket 분리 기준)
cust['first_delay_days'].describe(percentiles=(0.95, 0.99))

,first_delay_days
count,93349.000000
mean,-11.849982
std,10.183084
min,-147.000000
50%,-12.000000
95%,3.000000
99%,18.000000
max,188.000000


#### ✅ 극단값 처리
- min = -147 일
- 99% = 18 일
- max = 188 일

※ 분석 목적: 배송 지연이 재구매 행동에 영향을 주기 시작하는 구간의 탐색<br><br>
재구매 행동에 영향을 주는 일반적 배송 경험 구간 분석을 위해 <br>
전체의 99%를 벗어나는 극단 배송 사례는 별도 케이스로 판단, 본 분석에서는 제외

In [17]:
# 분석 목적에 따른 범위 설정
upper_cut = cust['first_delay_days'].quantile(0.99)
lower_cut = cust['first_delay_days'].quantile(0.01)

cust_trim = cust[
    (cust['first_delay_days'] >= lower_cut) &
    (cust['first_delay_days'] <= upper_cut)
].copy()

In [18]:
# 제외 후 분포 확인
display(cust_trim['first_delay_days'].describe(percentiles=(0.95, 0.99)))

,first_delay_days
count,91620.000000
mean,-12.046551
std,8.167625
min,-36.000000
50%,-12.000000
95%,2.000000
99%,12.000000
max,18.000000


#### ✅ bucket 설정
* ≤0d      : 정시/조기
* 1d       : 경미한 지연
* 2–3d     : 인지 가능한 지연 (CX 임계점 근처)
* 4–5d     : 명확한 지연
* 6d+      : 클레임/이탈 위험 구간

In [19]:
# bucket 생성
bins = [-999, 0, 1, 3, 5, 999]
labels = ['≤0d', '1d', '2–3d', '4–5d', '6d+']

cust_trim['delay_bucket'] = pd.cut(
    cust_trim['first_delay_days'],
    bins=bins,
    labels=labels
)

In [20]:
# bucket별 표본수 검증
cust_trim['delay_bucket'].value_counts().sort_index()

,count
delay_bucket,
≤0d,86203
1d,805
2–3d,1009
4–5d,878
6d+,2725


In [21]:
# bucket별 D+30, D+60 retention 계산
bucket_retention = (
    cust_trim
        .groupby('delay_bucket')
        .agg(
            n_customers=('customer_unique_id', 'nunique'),
            d30_rate=('repurchase_d30', 'mean'),
            d60_rate=('repurchase_d60', 'mean')
        )
        .reset_index()
)

bucket_retention['d30_rate_pct'] = bucket_retention['d30_rate'] * 100
bucket_retention['d60_rate_pct'] = bucket_retention['d60_rate'] * 100

display(bucket_retention)

/tmp/ipython-input-3246877593.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('delay_bucket')


,delay_bucket,n_customers,d30_rate,d60_rate,d30_rate_pct,d60_rate_pct
0,≤0d,86203,0.015255,0.018398,1.525469,1.839843
1,1d,805,0.016149,0.019876,1.614907,1.987578
2,2–3d,1009,0.005946,0.007929,0.594648,0.792864
3,4–5d,878,0.010251,0.011390,1.025057,1.138952
4,6d+,2725,0.014679,0.018349,1.467890,1.834862


#### ✅ Retention 임계점 탐색 결과
- `≤0d`, `1d` 그룹: D+30/60 재구매율 상대적으로 높음
- `2-3d` 그룹: 명확한 하락이 확인됨
- `4-5d` 그룹: `2-3d`보다 높은 재구매율
- `6d+` 그룹: `≤0d`와 유사한 수준의 재구매율
<br>

=> 배송 지연 2~3일 구간에서 재구매율이 가장 크게 하락<br>
이후 지연이 더 길어질수록 추가적인 하락은 제한적<br>
: 특히 6일 이상 지연 고객은 정시 배송 고객과 유사한 retention<br>
(중간 구간만 유독 낮은 U자형 패턴)<br><br>

=> 배송 지연에 민감한 고객군이 2~3일 구간에서 주로 영향을 받으며<br>
장기 지연 구간에는 사전 기대가 다른 고객군 또는 둔감한 고객만 남아있기 때문으로 해석 가능

## 📌 Revenue at Risk
> 비용 대비 효과가 가장 큰 구간 탐색 (유력 후보: 2-3일 지연)

- 시나리오 A: 2-3일(≥2d) 이상 지연을 관리
- 시나리오 B: 4-5일(≥4d) 이상 지연을 관리

#### ✅ 전제

- 기준 주문: 첫 구매 주문
- 기준 지표: D+30 재구매율
- AOV: 첫 구매 주문의 평균 결제금액
- Δretention: 기준선(≤1d) 대비 하락폭
- 해석: 추정치(assumption-based)

In [22]:
# 기준선 설정 (≤1d)
baseline_rate = (
    cust_trim
      .loc[cust_trim['delay_bucket'].isin(['≤0d', '1d']), 'repurchase_d30']
      .mean()
)

print(f"기준선 (≤1d) 기준 평균 D+30 retention: {baseline_rate:.3f}")

기준선 (≤1d) 기준 평균 D+30 retention: 0.015


#### ✅ 시나리오 A: 2–3d 이상 (≥2d)

In [23]:
# 대상 고객/주문 수
mask_A = cust_trim['delay_bucket'].isin(['2–3d', '4–5d', '6d+'])
n_customers_A = cust_trim.loc[mask_A, 'customer_unique_id'].nunique()

print(f"2일 지연 이상 그룹 고객수: {n_customers_A}")

2일 지연 이상 그룹 고객수: 4612


In [24]:
# 해당 구간 평균 재구매율
rate_A = cust_trim.loc[mask_A, 'repurchase_d30'].mean()
delta_A = baseline_rate - rate_A

print(f"기준선(≤1d)의 평균 재구매율: {baseline_rate:.3f}")
print(f"2일 이상 지연 그룹의 평균 재구매율: {rate_A:.3f}")
print(f"2일 이상 지연으로 인해 사라진 재구매 확률: {delta_A:.3f}")

기준선(≤1d)의 평균 재구매율: 0.015
2일 이상 지연 그룹의 평균 재구매율: 0.012
2일 이상 지연으로 인해 사라진 재구매 확률: 0.003


In [25]:
# 첫 구매 AOV
first_orders = df.merge(
    cust[['customer_unique_id', 'first_order_id']],
    left_on='order_id',
    right_on='first_order_id',
    how='inner'
)

aov_first = first_orders['payment_value'].mean()
print(f"첫 구매의 평균 주문 금액: {aov_first:.2f}")

첫 구매의 평균 주문 금액: 160.32


In [26]:
# Revenue at Risk 계산
# = 고객수 * 평균 주문 금액 * 재구매율 차이
revenue_at_risk_A = n_customers_A * aov_first * delta_A
print(f"2일 이상 지연으로 발생할 수 있는 손실 금액: {revenue_at_risk_A:.2f}")

2일 이상 지연으로 발생할 수 있는 손실 금액: 2467.83


#### ✅ 시나리오 B: 4–5d 이상(≥4d)

In [27]:
# 대상 고객/주문 수
mask_B = cust_trim['delay_bucket'].isin(['4–5d', '6d+'])
n_customers_B = cust_trim.loc[mask_B, 'customer_unique_id'].nunique()

print(f"4일 지연 이상 그룹 고객수: {n_customers_B}")

4일 지연 이상 그룹 고객수: 3603


In [28]:
# 해당 구간 평균 재구매율
rate_B = cust_trim.loc[mask_B, 'repurchase_d30'].mean()
delta_B = baseline_rate - rate_B

print(f"기준선(≤1d)의 평균 재구매율: {baseline_rate:.3f}")
print(f"4일 이상 지연 그룹의 평균 재구매율: {rate_B:.3f}")
print(f"4일 이상 지연으로 인해 사라진 재구매 확률: {delta_B:.3f}")

기준선(≤1d)의 평균 재구매율: 0.015
4일 이상 지연 그룹의 평균 재구매율: 0.014
4일 이상 지연으로 인해 사라진 재구매 확률: 0.002


In [29]:
# Revenue at Risk
revenue_at_risk_B = n_customers_B * aov_first * delta_B

print(f"4일 이상 지연으로 발생할 수 있는 손실 금액: {revenue_at_risk_B:.2f}")

4일 이상 지연으로 발생할 수 있는 손실 금액: 960.73


In [37]:
# 시나리오 A, B 비교
rar_summary = pd.DataFrame({
    'scenario': ['≥2d', '≥4d']
    , 'n_customers': [n_customers_A, n_customers_B]
    , 'delta_retention': [delta_A, delta_B]
    , 'estimated_RaR': [revenue_at_risk_A, revenue_at_risk_B]
})

display(rar_summary)

,scenario,n_customers,delta_retention,estimated_RaR
0,≥2d,4612,0.003338,2467.829003
1,≥4d,3603,0.001663,960.733325


#### ✅ 시나리오 비교 분석 결과
≥2일 배송 지연의 대상 고객 비중과 재구매율 하락폭(0.33%p)이 크기 때문에
<br>추정 매출 손실(2,468)이 크게 산출됨
<br><br>

≥4일 배송 지연의 대상 고객 고객 수도 적지 않지만(≥2일와 78% 차이)
<br>재구매율 하락폭(0.17%p)이 ≥2일 대비 1/2배 수준으로 상대적으로 작아
<br>추정 매출 리스크가 낮게 산출됨(961)

#### ✅ Bucket별 분석
- 시나리오 C: 2-3일 지연만 분리
- 시나리오 D: 4-5일 지연만 분리
- 시나리오 E: 6+일 지연만 분리

In [34]:
# 시나리오 C: 2–3d만
mask_C = cust_trim['delay_bucket'].isin(['2–3d'])
n_customers_C = cust_trim.loc[mask_C, 'customer_unique_id'].nunique()
rate_C = cust_trim.loc[mask_C, 'repurchase_d30'].mean()
delta_C = baseline_rate - rate_C
revenue_at_risk_C = n_customers_C * aov_first * delta_C

# 시나리오 D: 4–5d만
mask_D = cust_trim['delay_bucket'].isin(['4–5d'])
n_customers_D = cust_trim.loc[mask_D, 'customer_unique_id'].nunique()
rate_D = cust_trim.loc[mask_D, 'repurchase_d30'].mean()
delta_D = baseline_rate - rate_D
revenue_at_risk_D = n_customers_D * aov_first * delta_D

# 시나리오 E: 6d+만
mask_E = cust_trim['delay_bucket'].isin(['6d+'])
n_customers_E = cust_trim.loc[mask_E, 'customer_unique_id'].nunique()
rate_E = cust_trim.loc[mask_E, 'repurchase_d30'].mean()
delta_E = baseline_rate - rate_E
revenue_at_risk_E = n_customers_E * aov_first * delta_E

In [38]:
# 전체 시나리오 비교 분석
rar_detail = pd.DataFrame({
    'bucket': ['≥2d', '≥4d', '2–3d', '4–5d', '6d+']
    , 'n_customers': [n_customers_A, n_customers_B, n_customers_C, n_customers_D, n_customers_E]
    , 'rate_d30': [rate_A, rate_B, rate_C, rate_D, rate_E]
    , 'delta_vs_baseline': [delta_A, delta_B, delta_C, delta_D, delta_E]
    , 'RaR': [revenue_at_risk_A, revenue_at_risk_B, revenue_at_risk_C, revenue_at_risk_D, revenue_at_risk_E]
    })

# 고객 1명당 손실 계산
rar_detail['rar_per_customer'] = rar_detail['RaR'] / rar_detail['n_customers']

display(rar_detail)

,bucket,n_customers,rate_d30,delta_vs_baseline,RaR,rar_per_customer
0,≥2d,4612,0.011925,0.003338,2467.829003,0.535089
1,≥4d,3603,0.013600,0.001663,960.733325,0.266648
2,2–3d,1009,0.005946,0.009316,1507.095677,1.493653
3,4–5d,878,0.010251,0.005012,705.565714,0.803606
4,6d+,2725,0.014679,0.000584,255.167611,0.093639


#### ✅ 시나리오 종합 분석 결과 / ROI 우선순위

- RaR 기여도<br>
    : 2–3d RaR / ≥2d RaR <br>
    = 1,507 / 2,468<br>
    ≈ 61%
- 고객 한명당 리스크
    - 2–3d ≈ 1.49
    - 4–5d ≈ 0.80
    - 6d+ ≈ 0.09<br>

=> 6d+는 건수가 많지만 재구매율 하락폭이 작아 매출 리스크가 제한적<br>
반면, 2-3d 규모는 작아도 하락폭이 크기 때문에 고객당 리스크가 가장 크며 (1.493),<br>
전체 RaR의 61%를 차지하므로 ROI 우선순위 1순위